In [2]:
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy
import pandas as pd
from urllib.request import urlretrieve
from ucimlrepo import fetch_ucirepo 
import seaborn as sns

### Descent line search

In [ ]:
import numpy as np

def descentLineSearch(F, descent, ls, alpha0, x0, tol, maxIter, stopType):
    """
    Wrapper function executing descent with line search (includes inexact Newton;
    stopping with Newton decrement and grad)

    Parameters:
    F (dict): Dictionary with fields:
        - f: function handler
        - df: gradient handler
        - d2f: Hessian handler
    descent (str): Specifies descent direction {'steepest', 'newton', 'newton-cg', 'bfgs'}
    ls (function): Specifies the line search to apply
    alpha0 (float): Initial step length
    x0 (numpy.ndarray): Initial iterate
    tol (float): Stopping condition on minimal allowed step
    maxIter (int): Maximum number of iterations
    stopType (str): Stopping criteria choose from {'step', 'grad', 'dQx', 'NWdec'}

    Returns:
    xMin (numpy.ndarray): Minimum
    fMin (float): Value of f at the minimum
    nIter (int): Number of iterations
    info (dict): Dictionary with information about the iteration
        - xs: iterate history
        - alphas: step lengths history
        - NWdecs: Newton decrement history
        - Hk: Dictionary with Hessian approximation history (optional)
    """
    # Initialization
    nIter = 0
    x_k = x0
    info = {'xs': [x0], 'alphas': [alpha0], 'NWdecs': [], 'stopType': stopType}
    stopCond = False

    # Loop until convergence or maximum number of iterations
    while not stopCond and nIter <= maxIter:
        # Increment iterations
        nIter += 1

        # Compute descent direction
        if descent.lower() == 'steepest':
            p_k = -F['df'](x_k)  # steepest descent direction
        elif descent.lower() == 'newton':
            p_k = -np.linalg.solve(F['d2f'](x_k), F['df'](x_k))  # Newton direction
            if np.dot(p_k, F['df'](x_k)) > 0:
                print(f"Newton iteration {nIter}, produced not a descent direction")
        elif descent.lower() == 'newton-cg':
            # Conjugate gradient method
            df_k = F['df'](x_k)  # gradient
            B_k = F['d2f'](x_k)  # hessian
            eps_k = min(0.5, np.sqrt(np.linalg.norm(df_k))) * np.linalg.norm(df_k)
            z_j = np.zeros_like(df_k)
            r_j = df_k
            d_j = -df_k
            stopCondCG = False
            maxIterCG = 200  # maxIter
            nIterCG = 0
            while not stopCondCG and nIterCG <= maxIterCG:
                if np.dot(d_j, np.dot(B_k, d_j)) <= 0:
                    if nIterCG == 0:
                        p_k = d_j
                    else:
                        p_k = z_j
                    stopCondCG = True
                    eps_k = 0
                norm_r_j = np.dot(r_j, r_j)
                a_j = norm_r_j / np.dot(d_j, np.dot(B_k, d_j))
                z_j = z_j + a_j * d_j
                r_j = r_j + a_j * np.dot(B_k, d_j)
                if np.sqrt(np.dot(r_j, r_j)) < eps_k or nIterCG == maxIterCG:
                    stopCondCG = True
                    p_k = z_j
                b_j = np.dot(r_j, r_j) / norm_r_j
                d_j = -r_j + b_j * d_j
                nIterCG += 1
        else:
            raise ValueError("Invalid descent method specified.")

        # Call line search given by handle ls for computing step length
        alpha_k = ls(x_k, p_k, alpha0)

        # Newton decrement
        NWdec = alpha_k**2 * np.dot(p_k, np.dot(F['d2f'](x_k), p_k)) / 2
        info['NWdecs'].append(NWdec)

        # Update x_k and f_k
        x_k_1 = x_k
        x_k = x_k + alpha_k * p_k

        # Store iteration info
        info['xs'].append(x_k)
        info['alphas'].append(alpha_k)

        if stopType == 'step':
            # Compute relative step length
            normStep = np.linalg.norm(x_k - x_k_1) / np.linalg.norm(x_k_1)
            stopCond = (normStep < tol)
        elif stopType == 'grad':
            stopCond = (np.linalg.norm(F['df'](x_k), np.inf) < tol * (1 + abs(F['f'](x_k))))
        elif stopType == 'dQx':
            stopCond = (np.linalg.norm(F['df'](x_k), 2) < tol)
        elif stopType == 'NWdec':
            stopCond = NWdec < tol

    # Assign output values
    xMin = x_k
    fMin = F['f'](x_k)
    
    return xMin, fMin, nIter, info


### Interior Point Method PD

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from InteriorPoint.interiorPoint_PrimalDual import interiorPoint_PrimalDual
from InteriorPoint.mergeConstraints import mergeConstraints
from tutorial1.solutions.m.convergenceHistory import visualizeConvergence
from tutorial3.solutions.m.quartic1 import quartic1

# Close all figures and clear workspace
plt.close('all')
np.random.seed(0)
plt.style.use('ggplot')

# Objective function
F = quartic1()

# Constraints
# Inequality constraint (linear): y < x - 1 <=> -x + y + 1 < 0
constraintL = {'f': lambda x: -x[0] + x[1] + 1,
               'df': lambda x: np.array([-1, 1]),
               'd2f': lambda x: np.zeros((2, 2))}
ineqConstraints = [constraintL]  # {constraintC}, {constraintL}, {constraintC, constraintL},
lambda0 = np.ones(len(ineqConstraints))  # Ensure: lambda > 0

# Equality constraint
eqConstraint = {'A': np.array([[-2, -1]]), 'b': -1/2}
nu0 = 1

# Initialization
x0 = np.array([1, -2*1 + 1/2])

# Parameters
mu = 10  # in (3, 100);
tol = 1e-12
maxIter = 200
optsBT = {'maxIter': 30, 'alpha': 0.1, 'beta': 0.8}

# Minimization
ineqConstraint = mergeConstraints(ineqConstraints)
xPD, fPD, tPD, nPD, infoPD = interiorPoint_PrimalDual(F, ineqConstraint, eqConstraint, x0, lambda0, nu0, mu, tol, tol, maxIter, optsBT)
print('xPD =', xPD)

# Visualize iterates
x = np.arange(-8, 9, 0.1) / 4
y = np.arange(-8, 9, 0.1) / 4
X, Y = np.meshgrid(x, y)
Z = F['f'](X, Y)

visualizeConvergence(infoPD, X, Y, Z, 'final')
plt.title('Primal-Dual')
plt.xlabel('x')
plt.ylabel('y')
plt.plot(x, x - 1, '-.k', linewidth=2)
plt.plot(x, -2*x + 1/2, '--k', linewidth=2)
plt.legend(['f(x)', '$x_k$', '$y \leq x-1$', '$y = -2x+1/2$'], loc='upper right', fontsize='large')
plt.axis([x[0], x[-1], y[0], y[-1]])
plt.grid(True)

# Primal-Dual: residuals, surrogate gap
plt.figure()
plt.semilogy(infoPD['r_dual'], '-ob', markersize=5)
plt.semilogy(infoPD['r_cent'], '-+k', markersize=5)
plt.semilogy(infoPD['r_prim'] + np.finfo(float).eps, '--xr', markersize=5)
plt.semilogy(infoPD['surgap'], '--sg', markersize=5)
plt.legend(['Dual residual', 'Centering residual', 'Primal residual', 'Surrogate gap'])
plt.title('Primal-Dual')
plt.grid(True)
plt.show()


In [ ]:
def realLog(x):
    if x <= 0:
        ret = -np.inf
    else:
        ret = np.log(x)
    return ret

### Interior Point Barrier

In [ ]:
import numpy as np
from descentLineSearch import descentLineSearch
from feasibleNewton import feasibleNewton
from backtracking import backtracking

def interiorPoint_Barrier(F, phi, eqConstraint, x0, t, mu, tol, maxIter):
    """
    Barrier method for inequality constraints.

    Solvers problem with inequality constraints. Handling of equality constraints is not implemented.
    Calls backtracking.m and descentLineSearch.m (expects Newton decrement stopping criteria)

    Parameters:
    F (dict): Dictionary with fields:
        - f: function to minimize
        - df: gradient of function
        - d2f: Hessian of function
    phi (dict): Structure with fields:
        - f: barrier function
        - df: gradient of barrier function
        - d2f: Hessian of barrier function
    eqConstraint (dict): Linear equality constraint A x = b as structure with fields:
        - A: matrix for the equality constraints
        - b: vector for the equality constraints
        Set [] if no constraints
    x0 (numpy.ndarray): Initial iterate
    t (float): Barrier weight
    mu (float): Factor to increase t <- mu*t
    tol (float): Tolerance for the (duality gap ~ m/t) assumed scaled with 1/m, m #inequality constraints.
    maxIter (int): Maximum number of iterations

    Returns:
    xMin (numpy.ndarray): Minimum
    fMin (float): Value of f at the minimum
    t (float): Final barrier weight
    nIter (int): Number of iterations
    infoBarrier (dict): Dictionary with information about the iteration
        - xs: iterate history for x
        - xs_nw: iterate history for y
        - inIter: number of interior iterations
        - dGap: duality gap (scaled) 1/t
    """
    # Initialize
    nIter = 0
    stopCond = False
    x_k = x0

    # Parameters for centering step
    alpha0 = 1
    opts = {'c1': 1e-4, 'c2': 0.9, 'rho': 0.5}
    tolNewton = 1e-12
    maxIterNewton = 100

    infoBarrier = {'xs': [x0], 'xs_nw': [x0], 'alphas_nw': [alpha0], 'fs': [F['f'](x0)], 'inIter': [0], 'dGap': [1/t], 'NWdecs': []}

    # Outer iteration
    while not stopCond and nIter < maxIter:
        print('Iteration', nIter)
        # Create function handler for centering step
        G = {'f': lambda x: t*F['f'](x) + phi['f'](x),
             'df': lambda x: t*F['df'](x) + phi['df'](x),
             'd2f': lambda x: t*F['d2f'](x) + phi['d2f'](x)}

        # Line search function
        lsFun = lambda x_k, p_k, alpha0: backtracking(G, x_k, p_k, alpha0, opts)

        # Centering step - inner iteration, with Newton decrement stopping
        if not eqConstraint:
            x_k, f_k, nIterNW, infoNW = descentLineSearch(G, None, lsFun, alpha0, x_k, tolNewton, maxIterNewton, 'NWdec')
        else:
            x_k, f_k, nIterNW, infoNW = feasibleNewton(G, eqConstraint, lsFun, alpha0, x_k, tolNewton, maxIterNewton, 'NWdec')

        # Check stopping condition (m/t)
        if 1/t < tol:
            stopCond = True

        # Increase t
        t = mu * t

        # Store info
        infoBarrier['xs'].append(x_k)
        infoBarrier['xs_nw'].append(infoNW['xs'][:, 1:])
        infoBarrier['alphas_nw'].append(infoNW['alphas'][1:])
        infoBarrier['NWdecs'].append(infoNW['NWdecs'])
        infoBarrier['fs'].append(f_k)
        infoBarrier['inIter'].append(nIterNW)
        infoBarrier['dGap'].append(1/t)

        # Increment number of iterations
        nIter += 1

    # Assign values
    xMin = x_k
    fMin = F['f'](x_k)

    return xMin, fMin, t, nIter, infoBarrier

### Interior Point Primal Duel

In [ ]:
def interiorPoint_PrimalDual(F, ineqConstraint, eqConstraint, x0, lambda0, nu0, mu, tol, tolFeas, maxIter, opts):
    """
    Primal-dual interior point method for inequality constraints.

    Solves problem with inequality constraints (mandatory) and equality
    constraints (optional) using primal-dual interior point method.

    Parameters:
    F (dict): Dictionary with fields:
        - f: function to minimise
        - df: gradient of function 
        - d2f: Hessian of function 
    ineqConstraint (dict): Structure with fields:
        - f: vectorial linear function that sets the inequality constraints
        - df: Jacobian of the vectorial function f
    eqConstraint (dict): Structure with fields:
        - A: matrix for the equality constraints
        - b: vector for the equality constraints
    x0 (numpy.ndarray): Initial iterate
    lambda0 (numpy.ndarray): Initial inequality Lagrange parameters
    nu0 (numpy.ndarray): Initial equality Lagrange parameters
    mu (float): Factor for increasing t <- t*mu
    tol (float): Tolerance for the surrogate duality gap 
    tolFeas (float): Tolerance for primal and dual residuals
    maxIter (int): Maximum number of iterations
    opts (dict): Options for backtracking

    Returns:
    xMin (numpy.ndarray): Minimum
    fMin (float): Value of f at the minimum
    t (float): Final barrier weight
    nIter (int): Number of iterations
    infoPD (dict): Dictionary with information about the iteration 
        - xs: iterate history for x 
        - lambdas: iterate history for lambda
        - nus: iterate history for nu
        - r_dual: dual residual history
        - r_cent: centering residual history
        - r_prim: primal residual history
        - surgap: surrogate gap history
        - s: step length history
        - t: barrier weight history
        - BTs: backtracking iterations history
        - s_max: maximum step length history
    """
    # Dimensions
    n = len(x0)
    # Check if the inequality constraints are set
    if 'f' not in ineqConstraint:
        ineqConstraint['f'] = lambda x: -np.inf
    if 'df' not in ineqConstraint:
        ineqConstraint['df'] = lambda x: np.zeros((1, n))
    m = len(lambda0)
    # Check if the equality constraints are set
    if 'A' not in eqConstraint:
        eqConstraint['A'] = np.zeros((0, n))
    if 'b' not in eqConstraint:
        eqConstraint['b'] = np.zeros((0, 1))
    nEq = eqConstraint['A'].shape[0]

    # Initilize
    nIter = 0
    stopCond = False
    x_k = x0
    l_k = lambda0
    n_k = nu0
    infoPD = {'xs': [x0], 'lambdas': [lambda0], 'nus': [nu0], 'r_dual': [], 'r_cent': [], 'r_prim': [], 'surgap': [],
              's': [], 't': [], 'BTs': [], 's_max': []}

    # Define residual function
    def r_dual(t, x, l, n):
        return F['df'](x) + np.dot(ineqConstraint['df'](x).T, l) + np.dot(eqConstraint['A'].T, n)

    def r_cent(t, x, l, n):
        return -np.diag(l).dot(ineqConstraint['f'](x)) - np.ones((m, 1)) / t

    def r_prim(t, x, l, n):
        return eqConstraint['A'].dot(x) - eqConstraint['b']

    res = lambda t, x, l, n: np.vstack((r_dual(t, x, l, n), r_cent(t, x, l, n), r_prim(t, x, l, n)))

    # Surrogate gap
    eta = -(ineqConstraint['f'](x0)).dot(lambda0)
    t = mu * m / eta

    # Loop
    while not stopCond and nIter < maxIter:
        print('Iteration', nIter)

        # Compute residual
        res_k = res(t, x_k, l_k, n_k)

        # Find direction
        hessIneqConstraints_x_k = sum(l_k[j] * ineqConstraint['d2f'](x_k)[:, :, j] for j in range(m))

        deltaY = -np.linalg.solve(
            np.block([[F['d2f'](x_k) + hessIneqConstraints_x_k, ineqConstraint['df'](x_k).T, eqConstraint['A'].T],
                      [-np.diag(l_k).dot(ineqConstraint['df'](x_k)), -np.diag(ineqConstraint['f'](x_k)).flatten(),
                       np.zeros((m, nEq))],
                      [eqConstraint['A'], np.zeros((nEq, m)), np.zeros((nEq, nEq))]]), res_k)

        # Backtracking line search adapted to PD
        deltaX = deltaY[:n]  # delta x
        deltaL = deltaY[n:n + m]  # delta lambda
        deltaN = deltaY[n + m:]  # delta nu
        # Ensure: lambda + s * deltaL >= 0
        l_k_neg = l_k[deltaL < 0] / deltaL[deltaL < 0]  # identify negative deltaL
        if l_k_neg.size > 0:
            # ensure l_k + s*deltaL >= 0 (only needed for deltaL < 0)
            # equivalent to s <= -l_k/*deltaL and (-l_k/*deltaL) > 0
            s_max = min(1, 0.99 * (-l_k[deltaL < 0] / deltaL[deltaL < 0]))  # *0.99 to ensure lambda > 0
        else:
            s_max = 1
        # multiply with rho in (0,1)
        s = s_max
        # Ensure:
        #  all ineqConstraint.f(x_k + s*deltaX)) < 0
        #  sufficient reduction of ||r||: ||r(x_k + s*deltaX, l_k + s*deltaL, n_k + s*deltaN) <= (1-alpha*s) r(x_k, l_k, n_k) ||
        nIterBT = 0
        stopCondBT = False
        while not stopCondBT and nIterBT < opts['maxIter']:
            # reduce 's' until 
            #  all ineqConstraint.f(x_k + s*deltaX)) < 0  &
            #  ||r(x_k + s*deltaX, l_k + s*deltaL, n_k + s*deltaN) <= (1-alpha*s) r(x_k, l_k, n_k) ||
            # are satisfied
            if max(ineqConstraint['f'](x_k + s * deltaX)) < 0 and \
                    np.linalg.norm(res(t, x_k + s * deltaX, l_k + s * deltaL, n_k + s * deltaN)) <= \
                    (1 - opts['alpha'] * s) * np.linalg.norm(res_k):
                stopCondBT = True
            else:
                s = opts['beta'] * s
                nIterBT += 1

        # Update point
        x_k = x_k + s * deltaX
        l_k = l_k + s * deltaL
        n_k = n_k + s * deltaN

        # Surrogate gap
        eta = -(ineqConstraint['f'](x_k)).dot(l_k)
        t = mu * m / eta

        # Stopping criteria
        # Check if all residuals are below the tolerances
        if eta < tol and \
                np.linalg.norm(r_dual(t, x_k, l_k, n_k)) <= tolFeas and \
                np.linalg.norm(r_prim(t, x_k, l_k, n_k)) <= tolFeas:
            stopCond = True

        # Harvest info
        infoPD['xs'].append(x_k)
        infoPD['lambdas'].append(l_k)
        infoPD['nus'].append(n_k)
        infoPD['r_dual'].append(np.linalg.norm(r_dual(t, x_k, l_k, n_k)))
        infoPD['r_cent'].append(np.linalg.norm(r_cent(t, x_k, l_k, n_k)))
        infoPD['r_prim'].append(np.linalg.norm(r_prim(t, x_k, l_k, n_k)))
        infoPD['surgap'].append(eta)
        infoPD['s'].append(s)
        infoPD['t'].append(t)
        infoPD['BTs'].append(nIterBT)
        infoPD['s_max'].append(s_max)

        # Increment number of iterations
        nIter += 1

    # Assign values
    xMin = x_k
    fMin = F['f'](x_k)

    return xMin, fMin, t, nIter, infoPD


### Merge Constraints

In [ ]:
def mergeConstraints(ineqConstraints):
    """
    Merges together multiple inequality constraints into multidimensional
    constraint as used in interiorPoint_PrimalDual.m

    Parameters:
    ineqConstraints (list): List containing individual constraints as dictionaries each comprising fields:
        constraint['f']: constraint function constraint['f'](x) < 0
        constraint['df']: gradient of constraint function (as a row)
        constraint['d2f']: Hessian of constraint function 

    Returns:
    constraint (dict): Multidimensional constraint as a dictionary with fields:
        constraint['f']: column vector with constraint functions constraint['f'](x) < 0 as elements
        constraint['df']: matrix with gradients of constraint functions as rows
        constraint['d2f']: 3 dimensional tensor with Hessians of constraint functions indexed by the third component
    """
    # Define functions for constraint, gradient, and Hessian
    def constraint(x):
        return np.array([constraint_dict['f'](x) for constraint_dict in ineqConstraints])

    def gradConstraint(x):
        return np.vstack([constraint_dict['df'](x) for constraint_dict in ineqConstraints])

    def HessianConstraint(x):
        return np.dstack([constraint_dict['d2f'](x) for constraint_dict in ineqConstraints])

    # Construct the merged constraint dictionary
    ineqConstraint = {
        'f': constraint,
        'df': gradConstraint,
        'd2f': HessianConstraint
    }

    return ineqConstraint

### Create Barrier

In [ ]:
def createBarrier(ineqConstraints):
    """
    Creates a barrier function from R^n -> R

    Parameters:
    ineqConstraints (list): List containing individual constraints as dictionaries each comprising fields:
        constraint['f']: constraint function constraint['f'](x) < 0
        constraint['df']: gradient of constraint function (as a row)
        constraint['d2f']: Hessian of constraint function 

    Returns:
    phi (dict): Barrier function with fields:
        phi['f']: barrier function
        phi['df']: gradient of barrier function
        phi['d2f']: Hessian of barrier function
    """
    # Define functions for barrier, gradient, and Hessian
    def barrier(x):
        f = 0
        for constraint_dict in ineqConstraints:
            f += -np.real(np.log(-constraint_dict['f'](x)))
        return f

    def gradBarrier(x):
        df = np.zeros_like(x)
        for constraint_dict in ineqConstraints:
            df += -1 / constraint_dict['f'](x) * constraint_dict['df'](x).T
        return df

    def HessianBarrier(x):
        d2f = np.zeros((len(x), len(x)))
        for constraint_dict in ineqConstraints:
            d2f += 1 / (constraint_dict['f'](x) ** 2) * (constraint_dict['df'](x) @ constraint_dict['df'](x).T) + \
                   -1 / constraint_dict['f'](x) * constraint_dict['d2f'](x)
        return d2f

    # Construct the barrier function dictionary
    phi = {
        'f': barrier,
        'df': gradBarrier,
        'd2f': HessianBarrier
    }

    return phi


### Feasible Newton?

In [ ]:
def feasibleNewton(F, eqConstraint, ls, alpha0, x0, tol, maxIter, stopType):
    """
    Minimizes convex function subject to linear equality constraints.

    Parameters:
    F (dict): Dictionary with fields:
        - f: function handler
        - df: gradient handler
        - d2f: Hessian handler
    eqConstraint (dict): Dictionary with fields:
        - A: matrix for the equality constraints
        - b: vector for the equality constraints
    ls (function): Specifies the line search to apply
    alpha0 (float): Initial step length
    x0 (numpy.ndarray): Feasible initialization
    tol (float): Stopping condition on minimal allowed step
                norm(x_k - x_k_1)/norm(x_k) < tol;
    maxIter (int): Maximum number of iterations
    stopType (str): Stopping condition {'step', 'grad', 'dQx', 'NWdec'}

    Returns:
    xMin (numpy.ndarray): Minimum
    fMin (float): Value of f at the minimum
    nIter (int): Number of iterations
    info (dict): Dictionary with information about the iteration
        - xs: iterate history
        - alphas: step lengths history
    """
    # Parameters
    if stopType is None:
        stopType = 'NWdec'

    # Initialization
    nIter = 0
    x_k = x0
    info = {'xs': [x0], 'alphas': [alpha0], 'NWdecs': [], 'stopType': stopType}
    stopCond = False

    # Sizes
    p = eqConstraint['A'].shape[0]
    n = len(x0)

    # Loop until convergence or maximum number of iterations
    while not stopCond and nIter <= maxIter:
        # Increment iterations
        nIter += 1

        # Newton direction: (Delta x, w)
        p_k = np.linalg.solve(
            np.block([[F['d2f'](x_k), eqConstraint['A'].T],
                      [eqConstraint['A'], np.zeros((p, p))]]),
            np.block([[-F['df'](x_k)], [np.zeros(p)]]))

        # Newton direction Delta x only
        dx_k = p_k[:n]

        # Call line search given by handle ls for computing step length
        alpha_k = ls(x_k, dx_k, alpha0)

        # Newton decrement lambda
        NWdec = alpha_k**2 * np.dot(dx_k, np.dot(F['d2f'](x_k), dx_k)) / 2
        info['NWdecs'].append(NWdec)

        # Update x_k
        x_k_1 = x_k
        x_k = x_k + alpha_k * dx_k

        # Store iteration info
        info['xs'].append(x_k)
        info['alphas'].append(alpha_k)

        # Stopping condition
        if stopType == 'step':
            normStep = np.linalg.norm(x_k - x_k_1) / np.linalg.norm(x_k_1)
            stopCond = normStep < tol
        elif stopType == 'grad':
            stopCond = np.linalg.norm(F['df'](x_k), np.inf) < tol * (1 + np.abs(F['f'](x_k)))
        elif stopType == 'dQx':
            stopCond = np.linalg.norm(F['df'](x_k), 2) < tol
        elif stopType == 'NWdec':
            stopCond = NWdec < tol

    # Assign output values
    xMin = x_k
    fMin = F['f'](x_k)

    return xMin, fMin, nIter, info

## Tutorial 8

### Question 1

In [3]:
def feasibleNewton(F, eqConstraint, lsFun, alpha0, x0, tol, maxIter):
    """
    Minimizes convex function subject to linear equality constraints using the feasible Newton method.

    Parameters:
    F (dict): Dictionary containing function, gradient, and Hessian.
    eqConstraint (dict): Dictionary containing equality constraints.
    lsFun (function): Line search function.
    alpha0 (float): Initial step length.
    x0 (numpy.ndarray): Feasible initialization.
    tol (float): Stopping condition on minimal allowed step.
    maxIter (int): Maximum number of iterations.

    Returns:
    xMin (numpy.ndarray): Minimum point.
    fMin (float): Minimum value of the function.
    nIter (int): Number of iterations.
    info (dict): Information about the iteration.
    """
    # Initialization
    nIter = 0
    x_k = x0
    info = {'xs': [x0], 'alphas': [alpha0], 'NWdecs': []}
    stopCond = False

    # Loop until convergence or maximum number of iterations
    while (not stopCond and nIter <= maxIter):
        # Increment iterations
        nIter += 1

        # Newton direction
        p_k = np.linalg.solve(np.block([[F['d2f'](x_k), eqConstraint['A'].T],
                                         [eqConstraint['A'], np.zeros_like(eqConstraint['A'])]]),
                              np.hstack([-F['df'](x_k), np.zeros(eqConstraint['A'].shape[0])]))

        # Newton direction Delta x only
        dx_k = p_k[:len(x_k)]

        # Call line search function
        alpha_k = Functions.linesearch(x_k, dx_k, alpha0)

        # Newton decrement
        NWdec = alpha_k**2 * dx_k @ F['d2f'](x_k) @ dx_k / 2
        info['NWdecs'].append(NWdec)

        # Update x_k
        x_k_1 = x_k
        x_k = x_k + alpha_k * dx_k

        # Store iteration info
        info['xs'].append(x_k)
        info['alphas'].append(alpha_k)

        # Check stopping condition
        stopCond = np.linalg.norm(x_k - x_k_1) / np.linalg.norm(x_k_1) < tol

    # Assign output values
    xMin = x_k
    fMin = F['f'](x_k)
    return xMin, fMin, nIter, info


def backtracking(F, x_k, p_k, alpha0, lsOpts):
    """
    Backtracking line search.

    Parameters:
    F (dict): Dictionary containing function and gradient.
    x_k (numpy.ndarray): Current point.
    p_k (numpy.ndarray): Search direction.
    alpha0 (float): Initial step length.
    lsOpts (dict): Line search options.

    Returns:
    alpha_k (float): Step length.
    """
    rho = lsOpts['rho']
    c1 = lsOpts['c1']
    alpha_k = alpha0
    while F['f'](x_k + alpha_k * p_k) > F['f'](x_k) + c1 * alpha_k * F['df'](x_k) @ p_k:
        alpha_k *= rho
    return alpha_k


# Define the objective function and its derivatives
def f(x):
    return (x[0] - 2 * x[1])**2 + (x[0]**2 + 2)**2

def df(x):
    return np.array([10 * x[0] + 4 * x[0]**3 - 4 * x[1], -4 * (x[0] - 2 * x[1])])

def d2f(x):
    return np.array([[10 + 12 * x[0]**2, -4], [-4, 8]])

# Define the equality constraint
eqConstraint = {'A': np.array([[1, -1]]), 'b': 4}

# Initialization
x0 = np.array([-4, -8])
alpha0 = 1
tol = 1e-6
maxIter = 20

# Define the line search options
lsOptsNewton = {'rho': 0.9, 'c1': 1e-4}

# Define the line search function
lsFun = lambda x_k, p_k, alpha0: backtracking({'f': f, 'df': df}, x_k, p_k, alpha0, lsOptsNewton)

# Solve the optimization problem
xMin, fMin, nIter, info = feasibleNewton({'f': f, 'df': df, 'd2f': d2f}, eqConstraint, lsFun, alpha0, x0, tol, maxIter)


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 3 and the array at index 1 has size 4

### Question 2

In [ ]:
from scipy.optimize import minimize

# Objective function
def objective(x):
    a, b = 1, 1.5
    return (x[0] - a) ** 2 + (x[1] - b) ** 2 / 2 - 1

# Inequality constraints
def constraint_C(x):
    return x[0] ** 2 + x[1] ** 2 - 2

def constraint_L(x):
    return -x[0] + x[1] + 1

# Equality constraint
def eq_constraint(x):
    return 3 / 2 * x[0] - x[1] - 2

# Combine constraints
constraints = ({'type': 'ineq', 'fun': constraint_C},
               {'type': 'ineq', 'fun': constraint_L},
               {'type': 'eq', 'fun': eq_constraint})

# Initial guess
x0 = np.array([1/2, 3/2*1/2-2])

# Minimize the objective function subject to constraints using SLSQP method
result = minimize(objective, x0, method='SLSQP', constraints=constraints, tol=1e-12, options={'maxiter': 200})

# Print the result
print("Optimal solution:", result.x)

# Tutorial 9

### Question 1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Objective function
def objective(x, a, b):
    return (x[0] - a) ** 2 + (x[1] - b) ** 2 / 2 - 1

# Constraint function (circle)
def constraint(x, xC, yC, rC2):
    return (x[0] - xC) ** 2 + (x[1] - yC) ** 2 - rC2

# Solve using SciPy's minimize function
def solve(a, b, xC, yC, rC2):
    # Initial guess
    x0 = np.array([a, b])
    
    # Objective function with additional parameters (a, b)
    obj_func = lambda x: objective(x, a, b)
    
    # Constraint function with additional parameters (xC, yC, rC2)
    cons = ({'type': 'eq', 'fun': lambda x: constraint(x, xC, yC, rC2)})
    
    # Minimize the objective function subject to the constraint
    result = minimize(obj_func, x0, constraints=cons)
    
    return result

# Parameters
a = 1
b = 1.5
xC = 0
yC = 0
rC2 = 2

# Solve the problem
result = solve(a, b, xC, yC, rC2)

# Print the result
print("Optimal solution:", result.x)

### Augmented lagrangian

In [ ]:
import numpy as np
from scipy.optimize import minimize

def augmentedLagrangian(F, eqConstraint, x0, mu, nu0, tol, maxIter):
    nu = nu0
    nIter = 0
    x_k = x0
    x_k_1 = x0
    xs = [x0]
    nus = [nu0]
    fs = [F['f'](x0) + nu * eqConstraint['f'](x0) + mu / 2 * eqConstraint['f'](x0) ** 2]

    # Line search parameters
    alpha0 = 1
    opts = {'c1': 1e-4, 'c2': 0.9, 'rho': 0.9}

    # Stopping condition: norm(x_k - x_k_1) < tol
    stopCond = False

    while nIter < maxIter and not stopCond:
        nIter += 1

        # Save previous iterate
        x_k_1 = x_k

        # Augmented Lagrangian function
        FALag = {}
        FALag['f'] = lambda x: F['f'](x) + nu * eqConstraint['f'](x) + mu / 2 * eqConstraint['f'](x) ** 2
        FALag['df'] = lambda x: F['df'](x) + nu * eqConstraint['df'](x) + mu * eqConstraint['f'](x) * eqConstraint['df'](x)
        FALag['d2f'] = lambda x: F['d2f'](x) + nu * eqConstraint['d2f'](x) + mu * eqConstraint['df'](x) @ eqConstraint['df'](x).T + mu * eqConstraint['f'](x) * eqConstraint['d2f'](x)

        # Solve using constrained optimization method
        result = minimize(FALag['f'], x_k, jac=FALag['df'], hess=FALag['d2f'], method='Newton-CG', tol=tol, options=opts)
        x_k = result.x
        xs.append(x_k)

        # Update nu
        nu += mu * eqConstraint['f'](x_k)
        nus.append(nu)

        # Evaluate stopping condition
        stopCond = np.linalg.norm(x_k - x_k_1) < tol

        # Compute function value
        f_k = F['f'](x_k)
        fs.append(f_k)

    xMin = x_k
    fMin = F['f'](x_k)

    info = {'xs': np.array(xs), 'nus': np.array(nus), 'fs': np.array(fs)}
    return xMin, fMin, nIter, info

### Quadratic Penalty

In [ ]:
import numpy as np
from scipy.optimize import minimize

def quadraticPenalty(F, eqConstraint, x0, mu0, beta, tol, maxIter, alpha0, opts):
    mu = mu0
    tau = 10e3 * tol
    nIter = 0
    x_k = x0
    x_k_1 = x0
    stopCond = False

    xs = [x0]
    mus = [mu0]
    FQPen = {}
    FQPen['mu'] = mu0
    FQPen['f'] = lambda x: F['f'](x) + FQPen['mu'] / 2 * eqConstraint['f'](x) ** 2
    FQPen['df'] = lambda x: F['df'](x) + FQPen['mu'] * eqConstraint['f'](x) * eqConstraint['df'](x)
    FQPen['d2f'] = lambda x: F['d2f'](x) + FQPen['mu'] * eqConstraint['df'](x) @ eqConstraint['df'](x).T + FQPen['mu'] * eqConstraint['f'](x) * eqConstraint['d2f'](x)

    fs = [FQPen['f'](x0)]

    while nIter < maxIter and not stopCond:
        nIter += 1

        x_k_1 = x_k

        FQPen['mu'] = mu
        FQPen['f'] = lambda x: F['f'](x) + FQPen['mu'] / 2 * eqConstraint['f'](x) ** 2
        FQPen['df'] = lambda x: F['df'](x) + FQPen['mu'] * eqConstraint['f'](x) * eqConstraint['df'](x)
        FQPen['d2f'] = lambda x: F['d2f'](x) + FQPen['mu'] * eqConstraint['df'](x) @ eqConstraint['df'](x).T + FQPen['mu'] * eqConstraint['f'](x) * eqConstraint['d2f'](x)

        lsFun = lambda x_k, p_k, alpha0: backtracking(FQPen, x_k, p_k, alpha0, opts)
        result = minimize(FQPen['f'], x_k, jac=FQPen['df'], hess=FQPen['d2f'], method='Newton-CG', tol=tau, options={'maxiter': 100}, args=(), callback=None)

        x_k = result.x
        xs.append(x_k)

        f_k = F['f'](x_k)
        fs.append(f_k)

        mu *= beta
        mus.append(mu)

        stopCond = np.linalg.norm(x_k - x_k_1) < tol
        tau /= beta

    xMin = x_k
    fMin = F['f'](x_k)

    info = {'xs': np.array(xs), 'mus': np.array(mus), 'fs': np.array(fs)}
    return xMin, fMin, nIter, info